# 1- Importing necessary packages:

In [ ]:
import tensorflow as tf # type: ignore
import os,keras # type: ignore
from keras.utils import image_dataset_from_directory # type: ignore
from tensorflow.keras.preprocessing.image import img_to_array # type: ignore
from keras.layers import Conv2D, Add # type: ignore
from keras.callbacks import EarlyStopping # type: ignore
from PIL import Image # type: ignore

# 2- Data Preprocesing 

Remove (not RGB) images

In [ ]:
def remove_1d_images(path):
    
    # Change the current working directory to the specified path
    os.chdir(path)
    
    # Iterate over each file in the directory
    for filename in os.listdir():       

        # Open the image using the PIL library
        with Image.open(filename) as image:        
            # Convert the image to a NumPy array
            array = img_to_array(image)        
            # Get the number of channels in the image
            channels = array.shape[-1]     

        # Check if the number of channels is not equal to 3 (not an RGB image)
        if channels != 3:
            # If it's not an RGB image, remove the file
            os.remove(filename)

Remove images that have a width less than 600 or a height less than 300.

In [ ]:
def remove_low_res_images(path, cropped_width, cropped_height):
    
    # Change the current working directory to the specified path
    os.chdir(path)
    # Get the list of files in the directory
    file_list = os.listdir()
    
    # Iterate over each file in the directory
    for filename in file_list:
        
        # Open the image using the PIL library
        with Image.open(filename) as image: 
       
            # Get the width and height of the image
            width = image.size[0]
            height = image.size[1]           

        # Check if the width is less than 600 or height is less than 300
        if width < cropped_width or height < cropped_height:
            # If the image is low resolution, remove the file
            os.remove(filename)

Create training and testing datasets

In [ ]:
# Create the training dataset using image_dataset_from_directory
def create_datasets(Dataset_location, cropped_width, cropped_height,batch_size ,seed, validation_split):
    train_set = image_dataset_from_directory(
        Dataset_location,
        image_size=(cropped_width, cropped_height),   # Resize images to the specified dimensions
        validation_split=validation_split,            # Split 20% of the data for validation
        subset='training',                            # Use the training subset of the data
        seed=seed,                                    # Set a seed for reproducibility
        batch_size=batch_size,                        # Use a batch size of 32 for training
        label_mode=None                               # No labels are provided (unsupervised)
    )

    # Create the testing dataset using image_dataset_from_directory
    test_set = image_dataset_from_directory(
        Dataset_location,
        image_size=(cropped_width, cropped_height),   # Resize images to the specified dimensions
        validation_split=validation_split,            # Split 20% of the data for validation
        subset='validation',                          # Use the validation subset of the data
        seed=seed,                                    # Set a seed for reproducibility
        batch_size=batch_size,                        # Use a batch size of 32 for testing
        label_mode=None                               # No labels are provided (unsupervised)
    )
    
    return train_set, test_set

Data scaling:

In [ ]:
def scaling(input_image):
    input_image = input_image / 255.0
    return input_image

Convert the images from RGB into YUV, then take only the Y(Luminance) channel, and Crop image 

In [ ]:
# Define a function to process input features
def process_features(input, new_width, new_height):
    # Convert the input image to YUV color space
    input = tf.image.rgb_to_yuv(input)   
    # Get the index of the last axis (channels dimension)
    last_axis = len(input.shape) - 1  
    # Split the YUV channels
    y, u, v = tf.split(input, 3, axis=last_axis)
    
    # Resize the Y channel to the specified dimensions using the area method
    # resize the luminance (brightness) channel differently from the chrominance (color) channels to maintain certain visual characteristics.
    return tf.image.resize(y, [new_width, new_height], method="area")   

# Define a function to process target (output) features
def process_target(input):
    # Convert the input image to YUV color space
    input = tf.image.rgb_to_yuv(input)
    # Get the index of the last axis (channels dimension)
    last_axis = len(input.shape) - 1
    # Split the YUV channels
    y, u, v = tf.split(input, 3, axis=last_axis)
    
    # Return only the Y channel as the target
    return y

In [ ]:
def mapping_feature_with_target(train_set,test_set,cropped_width, cropped_height, upscale_factor):
    # Calculate the input dimensions after downscaling
    input_width = cropped_width // upscale_factor
    input_height = cropped_height // upscale_factor
    
    # Apply the processing functions to the training set
    train_set = train_set.map(lambda x: (process_features(x, input_width, input_height), process_target(x)))

    # Apply the processing functions to the testing set
    test_set = test_set.map(lambda x: (process_features(x, input_width, input_height), process_target(x)))

    return train_set,test_set

# CNN Archetecture:

Building the RDB block:

In [ ]:
def rdb_block(inputs, numLayers):

    # Get the number of channels in the input data
    channels = inputs.get_shape()[-1]

    # Initialize a list to store intermediate outputs in the block
    storedOutputs = [inputs]

    # Iterate through the specified number of Conv2D layers
    for _ in range(numLayers):
        # Concatenate the stored outputs along the channel axis
        localConcat = tf.concat(storedOutputs, axis=-1)

        # Apply a Conv2D layer with a 3x3 kernel, "same" padding, and ReLU activation
        out = Conv2D(filters=channels, kernel_size=3, padding="same", activation="relu")(localConcat)

        # Append the output of the current Conv2D layer to the list of stored outputs
        storedOutputs.append(out)

    # Concatenate all intermediate outputs along the channel axis
    finalConcat = tf.concat(storedOutputs, axis=-1)

    # Apply a pointwise Conv2D layer with a 1x1 kernel and ReLU activation
    finalOut = Conv2D(filters=channels, kernel_size=1, padding="same", activation="relu")(finalConcat)

    # Add the output of the pointwise Conv2D layer to the original input (residual connection)
    finalOut = Add()([finalOut, inputs])

    # Return the final output of the RDB block
    return finalOut

CNN layers construction:

In [ ]:
def Model(channels, upscale_factor):
    # Input layer to accept images with shape (height, width, channels)
    inputs = keras.Input(shape=(None, None, channels))

    # Initial convolutional layers for feature extraction
    X = Conv2D(64, 5, padding='same', activation='relu', kernel_initializer='Orthogonal')(inputs)

    X = Conv2D(64, 3, padding='same', activation='relu', kernel_initializer='Orthogonal')(X)
    # Residual Dense Block (RDB) feature extraction
    X = rdb_block(X, numLayers=3)

    # Further convolutional layers after RDB block
    X = Conv2D(32, 3, padding='same', activation='relu', kernel_initializer='Orthogonal')(X)
    X = rdb_block(X, numLayers=3)

    # the convolutional layers after the last RDB block 
    X = Conv2D(16, 3, padding='same', activation='relu', kernel_initializer='Orthogonal')(X)
    X = rdb_block(X, numLayers=3)

    # Final convolutional layer to generate high-resolution output
    X = Conv2D(channels * (upscale_factor**2), 3, padding='same', activation='relu', kernel_initializer='Orthogonal')(X)

    # Upsample using depth_to_space operation to get the final high-resolution output
    outputs = tf.nn.depth_to_space(X, upscale_factor)

    # Create and return the Keras Model
    return keras.Model(inputs, outputs)


# Main

In [ ]:
#dataset Path
Dataset_location = "Data"

# Define the desired format for the model file name
MoldelName = 'Super_Resolved_Model.keras'

# Set the number of channels in the image
channels = 1 #means we only pass the Y(Luminance) channel only to train the model

# Define the dimensions for cropping the image and passing the hyperparameters 
cropped_width = 600
cropped_height = 300
batch_size=32
seed=240
validation_split=0.2

# Set the upscale factor for the image
upscale_factor = 3

#epochs 
epochs_number = 100

In [ ]:

########################################## Preprocess ##########################################
remove_1d_images(Dataset_location)
remove_low_res_images(Dataset_location, cropped_width, cropped_height)
train_set, test_set = create_datasets(Dataset_location, cropped_width, cropped_height,batch_size ,seed, validation_split)
train_set = train_set.map(scaling)
test_set = test_set.map(scaling)
train_set, test_set = mapping_feature_with_target(train_set,test_set, cropped_width, cropped_height, upscale_factor)

In [ ]:
########################################## Model Train ##########################################
# Define early stopping callback to monitor training loss
early_stopping = EarlyStopping(monitor='loss', patience=10, min_delta=0.0001)

# Create an instance of your defined model
model = Model(channels, upscale_factor)        

# Compile the model using Adam optimizer and Mean Squared Error (MSE) loss
model.compile(optimizer='adam', loss='MSE')

# Display the full summary of the model architecture
model.summary(print_fn=print)

# Use f-string to insert the actual number of epochs into the format
model_file_name = MoldelName.format(epochs=epochs_number)

# Train the model
model.fit(train_set, epochs=epochs_number, callbacks=[early_stopping], validation_data=test_set)

# Save the model with the dynamically generated file name
model.save(model_file_name)